In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random

In [ ]:
def roll(probabilities):
    probs = probabilities.items()
    probs_sum = sum([prob[1] for prob in probs])
    probs = [(prob[0], prob[1] / probs_sum) for prob in probs]

    random_mass = random.random()
    mass = 0

    for prob in probs:
        mass += prob[1]
        if random_mass < mass:
            return prob

    return probs[-1]

In [ ]:
def generate_random_graph(vertices_number, edge_probability):
    graph = nx.Graph()
    graph.add_nodes_from(range(vertices_number))

    for u in range(vertices_number):
        for v in range(u+1, vertices_number):
            if random.random() < edge_probability:
                graph.add_edge(u, v)

    return graph

In [ ]:
def remove_edges_with_probability(graph, edge_remove_probability):
    edges_to_remove = []
    for edge in graph.edges():
        if random.random() < edge_remove_probability:
            edges_to_remove.append(edge)
    graph.remove_edges_from(edges_to_remove)

In [ ]:
VERTICES_NUMBER = 6
ADD_EDGE_PROBABILITY = 0.8
REMOVE_EDGE_PROBABILITY = 0.2

random_graph = generate_random_graph(VERTICES_NUMBER, ADD_EDGE_PROBABILITY)
nx.draw(random_graph, with_labels=True)
plt.show()

remove_edges_with_probability(random_graph, REMOVE_EDGE_PROBABILITY)
nx.draw(random_graph, with_labels=True)
plt.show()